In [1]:
import pandas as pd
import os
import datetime as dt

In [2]:
# Load processed routes
# note, change the filepath as needed
file_path = os.path.join(os.getcwd(), "..", "data", "processed_routes_output", f"routes_1.csv")
df = pd.read_csv(file_path)
df['node_id'] = df['node_id'] + 1  # reindex node ID to be indexed from 1 to match Younseo's formatting
df['date'] = df['date'].apply(lambda x: dt.date.fromisoformat(x))
df.head(2)

,order,vehicle_id,request_id,location_type,node_id,start_window,end_window,arrive_time,depart_time,date,num_vehicles
0,0,0,-1,depot_start,1765,NaN,NaN,NaN,NaN,2016-01-01,3
1,1,0,128,pickup,1316,2163.0,3963.0,2163.0,2463.0,2016-01-01,3


There is a unique set of routes for a day+num_vehicles. If you want the routes for vehicle_id's 0, 1, 2 for num_vehicles=3 on date 2016-01-01 use the get_routes function below. If you want the 4 vehicle fleet use the same function but set num_vehicles=4 (will return a dataframe with vehicle_id's 0, 1, 2, 3. get_routes returns the routes for a single VRP run (date, num_vehicles).

In [4]:
def get_routes(df, date, num_vehicles):
    result = df[(df['date']==date) & (df['num_vehicles']==num_vehicles)]
    return result

date = dt.date(2016, 1, 1)
num_vehicles = 3
routes = get_routes(df, date, num_vehicles)
print(routes['vehicle_id'].unique())

[0 1 2]


Below lets look at the route for vehicle_id=0.

In [5]:
routes[routes['vehicle_id']==0]

,order,vehicle_id,request_id,location_type,node_id,start_window,end_window,arrive_time,depart_time,date,num_vehicles
0,0,0,-1,depot_start,1765,NaN,NaN,NaN,NaN,2016-01-01,3
1,1,0,128,pickup,1316,2163.0,3963.0,2163.0,2463.0,2016-01-01,3
2,2,0,193,pickup,4614,3158.0,4958.0,3182.0,3482.0,2016-01-01,3
3,3,0,128,dropoff,383,3028.0,4828.0,3723.0,4023.0,2016-01-01,3
4,4,0,268,pickup,1495,4204.0,6004.0,4220.0,4520.0,2016-01-01,3
...,...,...,...,...,...,...,...,...,...,...,...
91,91,0,3908,pickup,445,84618.0,86418.0,84736.0,85036.0,2016-01-01,3
92,92,0,3908,dropoff,2412,85347.0,87147.0,85765.0,86065.0,2016-01-01,3
93,93,0,3935,pickup,666,85052.0,86852.0,86219.0,86519.0,2016-01-01,3
94,94,0,3935,dropoff,1990,85904.0,87704.0,87371.0,87671.0,2016-01-01,3


* order: 0 is the first location for this route (always of type "depot_start"), then 1, 2, ... to last location ("depot_end").
* vehicle_id: if num_vehicles=3 then fleet is of size 3, there will be vehicle_id's of 0, 1, 2 for this route.
* request_id: matches request id in requests_1.csv
* location_type: either "depot_start", "depot_end", "pickup" or "dropoff"
* node_id: matches nodes.csv
* start_window: start time window constraint (seconds past midnight)
* end_window: end window constraint (seconds past midnight)
* arrive_time: time the vehicle arrives at this location (seconds past midnight)
* depart_time: time the vehicle departs this location (arrive_time + 5 minutes). (seconds past midnight)
* date: date of this VRP run.
* num_vehicles: fleet size.